<a href="https://colab.research.google.com/github/PopaBiancaStefana/Numerical-Calculus/blob/main/CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab1

In [ ]:
import math
import numpy as np


def ex1():
    m = -1
    u = 10 ** m

    while (1 + u) != 1:
        m -= 1
        u = 10 ** m

    u = pow(10, m + 1)
    print("u = " + str(u))
    return u


def ex2():
    u = ex1()
    x = 1.0
    y = u
    z = u

    print("(x+y)+z = " + str((x + y) + z))
    print("x+(y+z) = " + str(x + (y + z)))

    a = 0
    while (a * y) * z == a * (y * z):
        a += 1

    print("a = ", a, ", y = ", y, ", z = ", z)
    print("(a * y) * z: ", (a * y) * z)
    print("a * (y * z): ", a * (y * z))



def generate_rand_matrix(n):
    return np.random.randint(10, size=(n, n))


def nearestPowerOf2(n):
    a = int(math.log2(n))

    if 2 ** a == n:
        return n

    return 2 ** (a + 1)


def pad_matrix(matrix):
    n, _ = matrix.shape
    next_pow_two = nearestPowerOf2(n)

    padding_val = next_pow_two - n
    padded = np.pad(matrix, pad_width=((0, padding_val), (0, padding_val)))

    return padded


def is_power_of_two(n):
    return math.log2(n).is_integer()


def remove_padding(matrix, initial_dim):
    matrix = matrix[:initial_dim, :initial_dim]
    return matrix


def split_matrix(matrix):

    row, col = matrix.shape
    row2, col2 = row // 2, col // 2
    return matrix[:row2, :col2], matrix[:row2, col2:], matrix[row2:, :col2], matrix[row2:, col2:]


def my_mult(A, B):
    n, _ = A.shape
    result = np.zeros((n, n))
    for i in range(0, n):
        for j in range(0, n):
            for k in range(0, n):
                result[i, j] += A[i, k] * B[k, j]
    return result


def strassen_algo(A, B, n_min):

    # Base case when blocks are 1x1
    if len(A) <= n_min:
        return my_mult(A, B)

    # We split the matrices into quadrants recursively
    a11, a12, a21, a22 = split_matrix(A)
    b11, b12, b21, b22 = split_matrix(B)

    # Computing the 7 products, recursively (p1, p2...p7)
    p1 = strassen_algo(a11 + a22, b11 + b22, n_min)
    p2 = strassen_algo(a21 + a22, b11, n_min)
    p3 = strassen_algo(a11, b12 - b22, n_min)
    p4 = strassen_algo(a22, b21 - b11, n_min)
    p5 = strassen_algo(a11 + a12, b22, n_min)
    p6 = strassen_algo(a21 - a11, b11 + b12, n_min)
    p7 = strassen_algo(a12 - a22, b21 + b22, n_min)

    # Computing the values of the 4 quadrants of the final matrix c
    c11 = p1 + p4 - p5 + p7
    c12 = p3 + p5
    c21 = p2 + p4
    c22 = p1 + p3 - p2 + p6

    # Concatenate C1, C2, C3, C4
    result = np.vstack((np.hstack((c11, c12)), np.hstack((c21, c22))))

    return result


def multiply(matrix1, matrix2):
    n, _ = matrix1.shape
    if not is_power_of_two(n):
        matrix1 = pad_matrix(matrix1)
        matrix2 = pad_matrix(matrix2)

    result = strassen_algo(matrix1, matrix2, 4)
    return remove_padding(result, n)


if __name__ == '__main__':
    x = np.array([[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [2, 2, 2, 2]])
    y = np.array([[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3], [2, 2, 2, 2]])

    z = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])
    m = np.array([[1, 1, 1], [2, 2, 2], [3, 3, 3]])

    z = generate_rand_matrix(200)
    m = z.copy()
    numpy_matrix_mult = np.matmul(m, m)

    print('Strassen Algo:', multiply(z, z))
    print('Numpy result:', numpy_matrix_mult)



Strassen Algo: [[3586. 3763. 3247. ... 3612. 3795. 3867.]
 [3918. 4262. 4046. ... 4152. 4063. 4092.]
 [4223. 4508. 4080. ... 4376. 4232. 4406.]
 ...
 [3993. 4249. 4136. ... 4158. 4074. 4122.]
 [4382. 4432. 4382. ... 4488. 4211. 4471.]
 [3833. 4090. 3884. ... 3905. 3723. 3833.]]
Numpy result: [[3586 3763 3247 ... 3612 3795 3867]
 [3918 4262 4046 ... 4152 4063 4092]
 [4223 4508 4080 ... 4376 4232 4406]
 ...
 [3993 4249 4136 ... 4158 4074 4122]
 [4382 4432 4382 ... 4488 4211 4471]
 [3833 4090 3884 ... 3905 3723 3833]]


# Lab2


In [ ]:
import random
from functools import reduce
from math import sqrt

import numpy as np
import scipy.linalg

from colorama import Fore
from copy import deepcopy

epsilon = 0


def is_positive_definite(matrix, n):
    for i in range(n):
        if matrix[i][i] < sum(matrix[i][j] for j in range(n) if j != i):
            return False
    return True


def is_symmetric(matrix, n):
    for i in range(n):
        for j in range(i + 1, n):
            if matrix[i][j] != matrix[j][i]:
                return False
    return True


def read_data_from_file(file_name):
    global epsilon
    matrix = []
    b = []
    with open(file_name, 'r') as file:
        epsilon = float(file.readline())
        n = int(file.readline())

        for _ in range(n):
            matrix.append([float(num) for num in file.readline().split(' ')])

        for _ in range(n):
            b.append([float(file.readline())])

    return epsilon, n, matrix, b


def generate_random_data():
    global epsilon
    epsilon = float(input('epsilon = '))
    n = int(input('n = '))

    matrix = [[0.0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            matrix[i][j] = round(random.uniform(0.1, 100.0), 2)
            matrix[j][i] = matrix[i][j]
        matrix[i][i] = round(sum(matrix[i][k] for k in range(n)) + 1.0, 2)

    b = [[round(random.uniform(0.1, 100.0), 2)] for _ in range(n)]
    print('Random matrix:', matrix)
    print('b:', b)
    if not is_symmetric(matrix, n) or not is_positive_definite(matrix, n):
        print(Fore.RED + 'Invalid Matrix.')
    return epsilon, n, matrix, b


def initialize():
    option = input('How do you want to introduce data? ')
    if option == '1':
        file_name = input('Please enter the name of the file: ')
        return read_data_from_file(file_name)
    elif option == '2':
        return generate_random_data()
    else:
        error_msg = 'Invalid option. Try 1 (reading from file) or 2 (generate random data)'
        raise ValueError(Fore.RED + error_msg)


def cholesky_decomposition(matrix, n):
    d = [0.0 for _ in range(n)]
    for p in range(n):
        d[p] = matrix[p][p] - sum(d[k] * matrix[p][k] ** 2 for k in range(p))
        for i in range(p + 1, n):
            if d[p] >= epsilon:
                matrix[i][p] = (matrix[i][p] - sum(
                    d[k] * matrix[i][k] * matrix[p][k] for k in range(p))) / d[p]

    return matrix, d


def compute_determinant(d):
    determinant_l = 1  # the product of the elements of the diagonal is 1 for triangular matrix
    determinant_lt = 1
    determinant_d = reduce((lambda l1, l2: l1 * l2), d)

    return determinant_l * determinant_d * determinant_lt


def forward_substitution(matrix, b, n):
    solution = [[b[0][0]]]
    for i in range(1, n):
        result = b[i][0]
        for j in range(i):
            result -= matrix[i][j] * solution[j][0]
        solution.append([result])

    return solution


def backward_substitution(matrix, b, n):
    solution = [[0.0] for _ in range(n)]
    solution[n - 1][0] = b[n - 1][0]

    for i in range(n - 2, -1, -1):
        result = b[i][0]
        for j in range(i + 1, n):
            result -= matrix[j][i] * solution[j][0]

        solution[i][0] = result

    return solution


def diag_matrix_sol(matrix, b, n):
    solution = [[0.0] for _ in range(n)]
    for i in range(n):
        if matrix[i] >= epsilon:
            solution[i] = [b[i][0] / matrix[i]]

    return solution


def find_solution(L, d, b, n):
    determinant = compute_determinant(d)
    if determinant <= epsilon:
        raise ValueError(Fore.RED + 'Determinant is zero. The Matrix must be non-singular')
    z = forward_substitution(L, b, n)
    y = diag_matrix_sol(d, z, n)
    x = backward_substitution(L, y, n)

    print('Solution:', x)
    return x


def lu_decomposition(matrix):
    _, L, U = scipy.linalg.lu(matrix)
    print(Fore.YELLOW + 'LU decomposition:')
    print('L:\n{}\n'.format(L))
    print('U:\n{}\n'.format(U))


def find_solution_with_lib(matrix, b):
    x = np.linalg.solve(matrix, b)
    print(Fore.YELLOW + 'System solution using library:')
    print('x:\n{}\n'.format(x))
    return x


def euclidian_norm(vector, n):
    result = 0
    for i in range(n):
        result += sqrt(vector[i][0] ** 2)

    return result


def my_mult(A, B, n):
    result = []
    for i in range(n):
        temp = 0
        for k in range(n):
            temp += A[i][k] * B[k][0]
        result.append([temp])
    return result


def subtraction(matrix1, matrix2, n):
    result = []
    for i in range(n):
        result.append([matrix1[i][0] - matrix2[i][0]])
    return result


def verify_solution(A_init, x, b, n):
    result = my_mult(A_init, x, n)
    result = subtraction(result, b, n)
    if euclidian_norm(result, n) < 10 ** (-8):
        print('Congratulations! Your solution is correct.')
    else:
        error_msg = "I'm sorry! Your solution is wrong, check out the problem with pen and paper."
        raise ValueError(Fore.RED + error_msg)


def bonus(A, d, n):
    for p in range(n):
        x = d[p] + sum(d[k] * A[p][k] * A[p][k] for k in range(p))
        if A[p][p] - x > 10 ** (-5):
            return 'Your decomposition is wrong!'
        for i in range(p + 1, n):
            if d[p] >= epsilon:
                x = A[i][p] * d[p] + sum(d[k] * A[i][k] * A[p][k] for k in range(p))
                if A[p][i] - x > 10 ** (-5):
                    return 'Your decomposition is wrong!'
    return 'Your decomposition is right!'


if __name__ == '__main__':
    try:
        epsilon, n, A, b = initialize()
        A_init = deepcopy(A)
        A, d = cholesky_decomposition(A, n)
        print('A=', A)
        print('d=', d)
        print(compute_determinant(d))
        x = find_solution(A, d, b, n)
        lu_decomposition(A_init)
        y = find_solution_with_lib(A_init, b)
        verify_solution(A_init, x, b, n)
        print(bonus(A, d, n))

    except ValueError as exception:
        print(exception)


ModuleNotFoundError: ignored

# Lab3

In [ ]:
import random
from math import sqrt
# from colorama import Fore
import scipy.linalg
import numpy as np
from copy import deepcopy

epsilon = 0


def read_data_from_file(file_name):
    global epsilon
    A = []
    s = []
    with open(file_name, 'r') as file:
        epsilon = float(file.readline())
        n = int(file.readline())

        for _ in range(n):
            A.append([float(num) for num in file.readline().split(' ')])

        for _ in range(n):
            s.append(float(file.readline()))

    return epsilon, n, np.array(A), s


def generate_random_data():
    global epsilon
    epsilon = float(input('epsilon = '))
    n = int(input('n = '))

    A = [[0.0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            A[i][j] = round(random.uniform(0.1, 100.0), 2)

    s = [round(random.uniform(0.1, 100.0), 2) for _ in range(n)]

    return epsilon, n, np.array(A), s


def initialize():
    option = input('type: 1 - read from file or 2 - generate random data ')
    if option == '1':
        file_name = input('Please enter the name of the file: ')
        return read_data_from_file(file_name)
    elif option == '2':
        return generate_random_data()
    else:
        error_msg = 'Invalid option. Try 1 (read from file) or 2 (generate random data)'
        raise ValueError(error_msg)


def matrix_vector_mult(A, s, n):
    result = []
    for i in range(n):
        temp = 0
        for k in range(n):
            temp += A[i][k] * s[k]
        result.append(temp)

    return np.array(result)


def householder_decomposition(A, n):
    u = [0.0 for _ in range(n)]
    Qt = np.array([[float(i == j) for j in range(n)] for i in range(n)])  # identity matrix

    # at each step we transform the r column of A

    for r in range(n - 1):
        # calculate Pr - reflection matrix
        # calculate beta
        sigma = sum(A[i][r] ** 2 for i in range(r, n))

        if sigma <= epsilon:  # A is singular
            break

        k = sqrt(sigma)
        if A[r][r] >= 0:
            k = -k

        beta = sigma - k * A[r][r]

        # calculate vector u
        u[r] = A[r][r] - k
        for i in range(r + 1, n):
            u[i] = A[i][r]

        # A = Pr * A
        # transform columns r+1 ... n of A
        for j in range(r + 1, n):
            y = sum(u[i] * A[i][j] for i in range(r, n)) / beta
            for i in range(r, n):
                A[i][j] -= y * u[i]

        # transform column r of A
        A[r][r] = k
        for i in range(r + 1, n):
            A[i][r] = 0

        """ #b = Pr * b
        y = sum(u[i] * b[i] for i in range(r, n)) / beta
        for i in range(r, n):
            b[i] -= y * u[i] """

        # Qt = Pr * Qt , same transforms as to A
        for j in range(n):
            y = sum(u[i] * Qt[i][j] for i in range(r, n)) / beta
            for i in range(r, n):
                Qt[i][j] -= y * u[i]

    return Qt, A


def backward_substitution(a, b, n):
    solution = [0.0 for _ in range(n)]
    solution[n - 1] = b[n - 1] / a[n - 1][n - 1]

    for i in range(n - 2, -1, -1):
        result = b[i]
        for j in range(i + 1, n):
            result -= a[i][j] * solution[j]

        solution[i] = result / a[i][i]

    return solution


def find_x_with_householder(A, n, b_init):
    Qt, R = householder_decomposition(A, n)
    Qtb = matrix_vector_mult(Qt, b_init, n)

    # solve system Rx = QTb
    return backward_substitution(R, Qtb, n)


def matrix_transpose(A, n):
    return np.array([[A[j][i] for j in range(n)] for i in range(n)])


def find_x_with_library(A, n, b):
    Q, R = scipy.linalg.qr(A_init)
    print("Q library: ", Q)
    print("R library: ", R, "\n")

    Qt = Q.transpose()
    Qtb = matrix_vector_mult(Qt, b, n)

    # solve system Rx = Qtb
    return backward_substitution(R, Qtb, n)


def euclidian_norm(vector):
    result = 0
    for i in range(len(vector)):
        result += sqrt(vector[i] ** 2)

    return result


def subtraction(vector1, vector2):
    result = []
    for i in range(len(vector1)):
        result.append(vector1[i] - vector2[i])
    return result


def solve_system_display_errors(A_init, n, b_init, s):
    A = deepcopy(A_init)

    x_QR = find_x_with_library(A, n, b)
    print("x_QR: ", x_QR)

    x_householder = find_x_with_householder(A, n, b_init)
    print("x_householder: ", x_householder, "\n")

    result_dif = euclidian_norm(subtraction(x_QR, x_householder))
    print("||x_QR - x_householder ||: ", result_dif, "\n")

    err1 = euclidian_norm(subtraction(matrix_vector_mult(A_init, x_householder, n), b_init))
    print("||A_init * x_householder - b_init||: ", err1, "\n")

    err2 = euclidian_norm(subtraction(matrix_vector_mult(A_init, x_QR, n), b_init))
    print("||A_init * x_QR - b_init||: ", err2, "\n")

    s_norm = euclidian_norm(s)
    err3 = euclidian_norm(subtraction(x_householder, s)) / s_norm
    print("||x_householder - s || / ||s||: ", err3, "\n")

    err4 = euclidian_norm(subtraction(x_QR, s)) / s_norm
    print("||x_QR - s || / ||s||: ", err4, "\n")

    if True in (el > 10 ** (-6) for el in [result_dif, err1, err2, err3, err4]):
        error_msg = "I'm sorry! Your solution is wrong, check out the problem with pen and paper."
        raise ValueError(error_msg)
    else:
        print('Congratulations! Your solution is correct.\n')


def ord_1_norm(A):
    # the 1-norm is the maximum of the absolute column sums
    return max(sum(abs(A[i][j]) for i in range(len(A[0]))) for j in range(len(A[0])))


def a_inverse(A_init, Qt, R, n):
    # verify if A is singular
    is_singular = list(abs(R[i][i]) < epsilon for i in range(n))

    if True in is_singular:
        print("The matrix is singular, we can't compute its inverse.")
        return 0

    A_inv_househ = deepcopy(A_init)
    for j in range(n):
        b = Qt[:, j]
        # solve system Rx = b
        x = backward_substitution(R, b, n)
        A_inv_househ[:, j] = x

    print("A inverse householder: \n", A_inv_househ, "\n")

    A_inv_bib = scipy.linalg.inv(A_init)
    print("A inverse with library: \n", A_inv_bib, "\n")

    print("||A_inv_householder - A_inv_bib||: ", ord_1_norm(A_inv_househ - A_inv_bib), "\n")
    return A_inv_househ


def multiply_r_q(R, Q):
    n = len(R[0])
    result = [[0.0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(n):
            for k in range(i, n):
                result[i][j] += R[i][k] * Q[k][j]
    return np.array(result)


def bonus(A, n, Qt, R):
    Ak = deepcopy(A)
    Ak_next = multiply_r_q(R, matrix_transpose(Qt, n))

    while ord_1_norm(Ak_next - Ak) >= epsilon:
        Qt, R = householder_decomposition(Ak, n)
        Ak = Ak_next
        Ak_next = multiply_r_q(R, matrix_transpose(Qt, n))

    return Ak_next


if __name__ == '__main__':
    try:
        epsilon, n, A, s = initialize()
        A_init = deepcopy(A)
        print("A : ", A)
        print("s: ", s, "\n")

        b = matrix_vector_mult(A, s, n)
        print('b:', b, "\n")

        Qt, R = householder_decomposition(A, n)
        print("Qt householder: ", Qt, "\n")
        print("R householder: ", R, "\n")

        solve_system_display_errors(A_init, n, b, s)

        a_inverse(A_init, Qt, R, n)

        Ak_next = bonus(A_init, n, Qt, R)
        print("!Bonus! - matricea Ak+1 este in forma Schur reala - superior Hessenberg si bloc diagonala (slide 16)."
              "\nAk+1 este asemenea cu matricea A, valorile proprii ale acesteia fiind usor de calculat : \n", Ak_next)

    except ValueError as exception:
        print(exception)


type: 1 - read from file or 2 - generate random data 2
epsilon = 0.000000001
n = 10
A :  [[56.19 36.23  8.58 43.47 66.89 41.43 76.94 52.81 29.36 78.86]
 [25.38 74.19 62.1  52.17 24.48 56.   48.49  5.86 29.83 75.38]
 [ 5.83  6.73 89.83 68.13 78.76 13.03  7.24 54.76 19.18 15.9 ]
 [88.77 64.11 51.2  93.65 26.9  43.    0.63 72.46 71.74 51.24]
 [51.52 25.56 86.89 29.43 34.46 80.81 83.07  6.56  4.99 85.15]
 [98.99 22.14 89.65 32.68  4.84 86.41 12.36 99.22 23.04 52.71]
 [75.3   7.9  54.73 35.46 47.41 51.42 32.17 43.38 67.45 32.25]
 [ 3.42 47.42 53.42 37.31 61.53 43.75 49.62 28.81 96.54 85.33]
 [88.92 65.98 23.81 96.38 43.22 99.65 22.51 60.61  6.04 59.33]
 [36.03 37.41 66.42  8.84 42.66 54.75 54.41 25.18 16.52 39.57]]
s:  [67.18, 68.96, 15.75, 82.52, 34.2, 35.14, 8.58, 81.14, 55.84, 97.98] 

b: [28050.3464 24852.3344 18178.3753 36261.2588 22905.6653 30095.4781
 23542.0503 27576.536  35093.1947 17468.2562] 

Qt householder:  [[-0.2843265  -0.1284251  -0.02950033 -0.44918426 -0.26069588 -0.50089

# Lab4

In [ ]:
from colorama import Fore

epsilon = 1e-9
k_max = 10000


def display_data_structure(data_structure):
    for i, elem in enumerate(data_structure):
        print(i, elem)


def exists(tuple_list, poz):
    for i, tuple_elem in enumerate(tuple_list):
        if tuple_elem[1] == poz:
            return i
    return -1


def initialize_matrix_a(file_path):
    with open(file_path, 'r') as file:
        dimension = int(file.readline())
        vector = [[] for _ in range(dimension)]
        for file_line in file:
            value, i, j = [num.strip() for num in file_line.split(',')]
            value = float(value)
            i = int(i)
            j = int(j)
            poz = exists(vector[i], j)
            if poz != -1:
                vector[i][poz][0] += value
            else:
                vector[i].append([value, j])
        return vector, dimension


def initialize_vector_b(file_path):
    with open(file_path, 'r') as file:
        dimension = int(file.readline())
        vector = []
        for _ in range(dimension):
            vector.append([float(file.readline())])
        return vector


def verify_zero_elements(vector):
    for i, line in enumerate(vector):
        for tuple_elem in line:
            if i == tuple_elem[1]:
                if abs(tuple_elem[0]) < epsilon:
                    print(
                        Fore.RED + 'Attention! There are some null elements on principle diagonal')


def compute_sum_from_formula(list_of_tuples, line, x):
    result = 0.0
    elem_diag = 0.0
    for tuple_elem in list_of_tuples:
        if tuple_elem[1] != line:
            result += (tuple_elem[0] * x[tuple_elem[1]])
        else:
            elem_diag = tuple_elem[0]
    return result, elem_diag


def gauss_seidel(vector, b, n):
    k = 0
    x = [0.0] * n
    delta_x = 0.1
    iteration = 0
    while epsilon <= delta_x <= 10 ** 8 and k <= k_max:
        for i in range(n):
            sum_of_elem, elem_diag = compute_sum_from_formula(vector[i], i, x)
            element = (b[i][0] - sum_of_elem) / elem_diag
            delta_x = abs(x[i] - element)
            x[i] = element
            iteration += 1
        k += 1
    print(Fore.GREEN + f'The total number of iterations: {iteration}' + Fore.WHITE)
    if delta_x < epsilon:
        return x
    raise ValueError(Fore.RED + 'Does not converge!' + Fore.WHITE)


def infinity_norm(vector):
    return max([abs(element) for element in vector])


def multiply(list_of_tuples, x):
    result = 0.0
    for tuple_elem in list_of_tuples:
        result += tuple_elem[0] * x[tuple_elem[1]]
    return result


def verify_solution(vector, x, b):
    result = []
    for i, list_of_tuples in enumerate(vector):
        solution = multiply(list_of_tuples, x)
        result.append(solution - b[i][0])
    norm = infinity_norm(result)
    print(Fore.GREEN + 'Infinity norm = ' + str(norm) + Fore.WHITE)


def sum_of(a, b):
    actual_results, _ = initialize_matrix_a('files/aplusb.txt')
    for i, list_of_tuples in enumerate(a):
        for tuple_elem in list_of_tuples:
            poz = exists(b[i], tuple_elem[1])
            temp_result = 0
            if poz != -1:
                temp_result = b[i][poz][0]
            temp_result += tuple_elem[0]
            poz = exists(actual_results[i], tuple_elem[1])
            if poz != -1:
                if abs(temp_result - actual_results[i][poz][0]) >= epsilon:
                    error_msg = f'Incorrect result. Actual={temp_result} Expected={actual_results[i][poz][0]}'
                    raise ValueError(Fore.RED + error_msg + Fore.WHITE)
    print(Fore.GREEN + 'Sum is correct!' + Fore.WHITE)

def bonus():
    a, _ = initialize_matrix_a('files/a.txt')
    b, _ = initialize_matrix_a('files/b.txt')
    sum_of(a, b)


if __name__ == '__main__':
    A, n = initialize_matrix_a('a_1.txt')
    b = initialize_vector_b('b_1.txt')
    verify_zero_elements(A)
    try:
        x = gauss_seidel(A, b, n)
        display_data_structure(x)
        verify_solution(A, x, b)
        bonus()
    except ValueError as err:
        print(err)


ValueError: ignored

# Lab5

In [ ]:
from colorama import Fore
import numpy as np
import random
from math import sqrt

epsilon = 1e-9
k_max = 100000


def afisare(data_structure):
    for i, line in enumerate(data_structure):
        print(i, line)


def get_element(tuple_list, poz):
    for i, tuple_elem in enumerate(tuple_list):
        if tuple_elem[1] == poz:
            return tuple_elem[0], i
    return None, None


def initializare_matrice_a(file_path):
    with open(file_path, 'r') as file:
        dimension = int(file.readline())
        matrice = [[] for _ in range(dimension)]
        for file_line in file:
            value, i, j = [num.strip() for num in file_line.split(',')]
            value = float(value)
            i = int(i)
            j = int(j)
            el, poz = get_element(matrice[i], j)
            if poz is not None:
                matrice[i][poz][0] += round(value, 10)
            else:
                matrice[i].append([round(value, 10), j])
        return matrice, dimension


def get_element_2(valori, ind_col, valoare, indice):
    for i, el in enumerate(valori):
        if el == valoare and ind_col[i] == indice:
            return i
    return -1


def initializare_matrice_var2(path):
    valori = []
    ind_col = []
    inceput_linii = []

    with open(path, 'r') as file:
        n = int(file.readline())

        k = 1
        ultima_linie = -1
        for file_line in file:
            value, i, j = [num.strip() for num in file_line.split(',')]
            value = float(value)
            i = int(i)
            j = int(j)

            indice_aparitie = get_element_2(valori, ind_col, value, j)
            if indice_aparitie != -1:
                valori[indice_aparitie] += round(value, 10)
            else:
                valori.append(round(value, 10))
                ind_col.append(j)

            if ultima_linie != i:
                inceput_linii.append(k)
                ultima_linie = i
            k += 1
        inceput_linii.append(len(ind_col) + 1)

    return n, valori, ind_col, inceput_linii


def generare_matrice_rara_simetrica(n, densitate=0.2):
    matrice = [[] for _ in range(n)]

    for i in range(n):
        for j in range(i, n):
            value = 0
            while value == 0:
                value = round(random.uniform(-100, 100), 10)
            if i == j:
                matrice[i].append([value, j])
            elif random.random() < densitate and i != j:
                matrice[i].append([value, j])
                matrice[j].append([value, i])

    return matrice


def este_simetrica(v):
    for i, line in enumerate(v):
        for tuple_elem in line:
            if tuple_elem[1] > i:  # coloana > linia <=> deasupra diagonalei principale
                el, col = get_element(v[tuple_elem[1]], i)
                if tuple_elem[0] != el:
                    raise ValueError(Fore.RED + 'Atentie! Matricea nu este simetrica.' + Fore.WHITE)


def norma_euclidiana(vector):
    result = 0
    for i in range(len(vector)):
        result += vector[i] ** 2
    return sqrt(result)


def matrice_rara_vector_mult(A, v, n):
    result = []
    for i, line in enumerate(A):
        temp = 0
        for tuple_elem in line:
            temp += tuple_elem[0] * v[tuple_elem[1]]
        result.append(temp)
    return np.array(result)


def matrice_rara_vector_mult2(valori, ind_col, inceput_linii, v, n):
    result = []
    for i in range(n):
        temp = 0
        for j in range(inceput_linii[i] - 1, inceput_linii[i + 1] - 1):
            temp += valori[j] * v[ind_col[j]]
        result.append(temp)
    return np.array(result)


def generare_vector(n):
    return np.array([round(random.uniform(-100, 100.0), 10) for _ in range(n)])


def metoda_puterii(A, n):
    v = generare_vector(n)
    v = v / norma_euclidiana(v)
    w = matrice_rara_vector_mult(A, v, n)
    val_proprie = w.dot(v)
    k = 0

    while norma_euclidiana(w - val_proprie * v) > (n * epsilon) and k <= k_max:
        v = w / norma_euclidiana(w)  # normalizare
        w = matrice_rara_vector_mult(A, v, n)
        val_proprie = w.dot(v)
        k += 1

    if k > k_max:
        raise ValueError(Fore.RED + 'Nu converge, mareste valoarea lui epsilon!' + Fore.WHITE)
    else:
        return val_proprie, v


def metoda_puterii_var2(valori, ind_col, inceput_linii, n):
    v = generare_vector(n)
    v = v / norma_euclidiana(v)
    w = matrice_rara_vector_mult2(valori, ind_col, inceput_linii, v, n)
    val_proprie = w.dot(v)
    k = 0

    while norma_euclidiana(w - val_proprie * v) > (n * epsilon) and k <= k_max:
        v = w / norma_euclidiana(w)  # normalizare
        w = matrice_rara_vector_mult2(valori, ind_col, inceput_linii, v, n)
        val_proprie = w.dot(v)
        k += 1

    if k > k_max:
        raise ValueError(Fore.RED + 'Nu converge, mareste valoarea lui epsilon!' + Fore.WHITE)
    else:
        return val_proprie, v


""" # functii pentru numpy
def rang_matrice(valori_singulare):
    return len([valoare for valoare in valori_singulare if valoare > epsilon])

def numar_de_conditionare(valori_singulare):
    return max(valori_singulare) / min([valoare for valoare in valori_singulare if valoare > epsilon])
"""


def generare_matrice_clasica(p, n):
    A = [[0.0 for _ in range(n)] for _ in range(p)]
    for i in range(p):
        for j in range(n):
            A[i][j] = round(random.uniform(-100, 100.0), 10)
    return np.array(A)


def calculare_si(S, p, n):
    SI = [[0.0 for _ in range(p)] for _ in range(n)]
    val_poz = np.array([el for el in S if el > epsilon])
    rang = len(val_poz)

    for i in range(rang):
        SI[i][i] = 1 / val_poz[i]

    return np.array(SI)


def transpusa(A, p, n):
    transpose = np.zeros((n, p))
    for i in range(p):
        for j in range(n):
            transpose[j][i] = A[i][j]
    return np.array(transpose)


def norma_ord_1(A, p, n):
    # the 1-norm is the maximum of the absolute column sums
    return max(sum(abs(A[i][j]) for i in range(p)) for j in range(n))


def evalueaza_matrice_clasica(A, p, n):
    print(A)
    U, S, VT = np.linalg.svd(A, full_matrices=True)
    print('Valori singulare:\n ', S)

    rang = len([el for el in S if el > epsilon])  # nr de valori singulare pozitive
    print('Rang: ', rang)

    numar_conditionare = np.linalg.cond(np.diag(S))  # max valori singulare / min val singulare pozitive
    print('Numar de conditionare: ', numar_conditionare)

    SI = calculare_si(S, p, n)
    print("SI: ", SI)
    Ai = np.matmul(np.matmul(transpusa(VT, n, n), SI), transpusa(U, p, p))
    print('\nPseudoinversa Moore-Penrose: \n', Ai)

    At = transpusa(A, p, n)
    Aj = np.matmul(np.linalg.inv(np.matmul(At, A)), At)
    print('\nPseudoinversa in sensul celor mai mici patrate: \n', Aj)

    print('\nNorma matriceala 1: ', np.linalg.norm(Ai - Aj))



if __name__ == '__main__':
    p = int(input("Intordu vaoarea lui p: "))
    n = int(input("Intordu vaoarea lui n: "))

    try:
        if n > p:
            raise ValueError(Fore.RED + 'p trebuie sa fie >= n' + Fore.WHITE)
        if p == n:
            A1, n = initializare_matrice_a('m_rar_sim_2023_512.txt')
            este_simetrica(A1)
            valoare_proprie, vector_propriu = metoda_puterii(A1, n)
            print("Vector propriu pentru matircea A cu n= 512: \n", vector_propriu,
                  "\nValoare proprie pentru matircea A cu n= 512: ", valoare_proprie)

            A2, n = initializare_matrice_a('m_rar_sim_2023_1024.txt')
            este_simetrica(A2)
            valoare_proprie, vector_propriu = metoda_puterii(A2, n)
            print("\nVector propriu pentru matircea A cu n= 1024: \n", vector_propriu,
                  "\nValoare proprie pentru matircea A cu n= 1024: ", valoare_proprie)

            A3, n = initializare_matrice_a('m_rar_sim_2023_2023.txt')
            este_simetrica(A3)
            valoare_proprie, vector_propriu = metoda_puterii(A3, n)
            print("\nVector propriu pentru matircea A cu n= 2023: \n", vector_propriu,
                  "\nValoare proprie pentru matircea A cu n= 2023: ", valoare_proprie)

            A_random = generare_matrice_rara_simetrica(p)
            este_simetrica(A_random)
            valoare_proprie, vector_propriu = metoda_puterii(A_random, p)
            print("\nVector propriu pentru matricea A generata aleator cu n= ", p , " :\n", vector_propriu,
                  "\nValoare proprie pentru matricea A generata aleator cu n= ", p , ": ", valoare_proprie)
        else:
            A = generare_matrice_clasica(p, n)
            evalueaza_matrice_clasica(A, p, n)
    except ValueError as err:
        print(err)


Intordu vaoarea lui p: 992
Intordu vaoarea lui n: 992
Vector propriu pentru matircea A cu n= 512: 
 [ 9.96806178e-01 -2.17339594e-54 -9.38599423e-03  5.27297221e-49
 -3.43358784e-04 -7.34429049e-46 -3.82160523e-03  8.00086986e-44
 -8.22816498e-03 -3.51203034e-42 -6.54907464e-03 -5.10211245e-41
 -1.09896774e-02 -1.97424409e-39 -8.62386951e-03 -3.35040132e-38
 -4.66096738e-03  1.27377563e-37 -3.35781464e-03  1.20873851e-36
 -1.02524444e-02 -5.36108381e-36 -1.11273138e-02 -1.60315272e-35
 -1.77902313e-04  6.97387760e-35 -7.96899865e-03 -6.37706966e-34
 -1.26591951e-33 -1.16849124e-33 -2.85229387e-03  1.30340158e-33
 -3.70007526e-03 -1.20594012e-32 -8.08650203e-03  2.62949488e-32
 -2.32114931e-03 -8.00563104e-32 -1.03285836e-02 -3.35768551e-31
 -3.39478610e-03 -5.16035991e-31 -7.07582655e-04 -3.21326452e-30
 -1.59379973e-03  1.73772493e-30 -2.13043510e-03  1.45779614e-29
 -1.14757973e-03 -4.70807519e-30 -1.05097804e-02 -6.19166272e-29
 -5.67200892e-03 -4.61242719e-29 -2.17509371e-03 -2.227